In [41]:
import re


In [4]:
import pandas as pd

# Step 1: Load the Twitter dataset
twitter_df = pd.read_csv("Tweeter_data.csv")

# Step 2: View column names
print("Original Columns:")
print(twitter_df.columns.tolist())



Original Columns:
['css-175oi2r href', 'css-9pa8cd src', 'css-1jxf684', 'css-1jxf684 2', 'css-1jxf684 3', 'css-146c3p1', 'css-1jxf684 4', 'css-1jxf684 5', 'css-1jxf684 6', 'css-1jxf684 8', 'css-1jxf684 9', 'css-1jxf684 11', 'css-1jxf684 12', 'css-1jxf684 13', 'css-1jxf684 19', 'css-146c3p1 4', 'css-1jxf684 href 3', 'css-1jxf684 25', 'css-1jxf684 26', 'css-1jxf684 href 4', 'css-1jxf684 27']


In [2]:
# Step 3: Display first few rows
print("\nSample Rows:")
print(twitter_df.head(3))



Sample Rows:
            css-175oi2r href  \
0   https://x.com/BlendrNews   
1    https://x.com/valdombre   
2  https://x.com/Tablesalt13   

                                      css-9pa8cd src    css-1jxf684  \
0  https://pbs.twimg.com/profile_images/178314674...     BlendrNews   
1  https://pbs.twimg.com/profile_images/188231606...  Riley Donovan   
2  https://pbs.twimg.com/profile_images/192265124...      Tablesalt   

  css-1jxf684 2 css-1jxf684 3 css-146c3p1  \
0   @BlendrNews             ·      May 14   
1    @valdombre             ·       Jun 8   
2  @Tablesalt13             ·      Jun 26   

                                   css-1jxf684 4 css-1jxf684 5  \
0  Canada faces an economic crossroads:\r\n\r\n-  Unemployment   
1                                       Canada's  unemployment   
2                                       BREAKING      CANADIAN   

                                       css-1jxf684 6 css-1jxf684 8  ...  \
0  rose to 6.9%\r\n- 15.4% of young men are joble..

In [3]:
# Step 4: Check for missing values
print("\nMissing Values:")
print(twitter_df.isnull().sum())



Missing Values:
css-175oi2r href         1
css-9pa8cd src           1
css-1jxf684              4
css-1jxf684 2            0
css-1jxf684 3            1
css-146c3p1              1
css-1jxf684 4          123
css-1jxf684 5           54
css-1jxf684 6          172
css-1jxf684 8          671
css-1jxf684 9         1067
css-1jxf684 11          24
css-1jxf684 12         180
css-1jxf684 13         328
css-1jxf684 19        1170
css-146c3p1 4          616
css-1jxf684 href 3     573
css-1jxf684 25         552
css-1jxf684 26        1147
css-1jxf684 href 4    1097
css-1jxf684 27        1098
dtype: int64


In [7]:
print(twitter_df.isnull().mean()*100)

css-175oi2r href       0.064516
css-9pa8cd src         0.064516
css-1jxf684            0.258065
css-1jxf684 2          0.000000
css-1jxf684 3          0.064516
css-146c3p1            0.064516
css-1jxf684 4          7.935484
css-1jxf684 5          3.483871
css-1jxf684 6         11.096774
css-1jxf684 8         43.290323
css-1jxf684 9         68.838710
css-1jxf684 11         1.548387
css-1jxf684 12        11.612903
css-1jxf684 13        21.161290
css-1jxf684 19        75.483871
css-146c3p1 4         39.741935
css-1jxf684 href 3    36.967742
css-1jxf684 25        35.612903
css-1jxf684 26        74.000000
css-1jxf684 href 4    70.774194
css-1jxf684 27        70.838710
dtype: float64


In [6]:
# Step 5: Count duplicates
print("\nDuplicate Rows:", twitter_df.duplicated().sum())


Duplicate Rows: 62


In [8]:
# Step 6: Drop duplicates
twitter_cleaned = twitter_df.drop_duplicates()

In [10]:
# Step 7: Drop columns with >70% missing values
threshold = len(twitter_cleaned) * 0.7
twitter_cleaned = twitter_cleaned.dropna(thresh=threshold, axis=1)


In [ ]:
"""# Step 8: Rename only important columns
twitter_df = twitter_df.rename(columns={
    'css-1jxf684 4': 'tweet_text',           # Main tweet text
    'css-146c3p1': 'date',                   # Posting date
    'css-1jxf684 8': 'likes',                # Like count
    'css-1jxf684 11': 'retweets',            # Retweet count
    'css-1jxf684 5': 'keyword_from_tweets'   # Topic/keyword
})
"""

In [37]:
# Step 8: Combine multiple text columns to get full tweet
# Many parts of tweets are spread across css-1jxf684 columns
text_columns = [col for col in twitter_df.columns if 'css-1jxf684' in col]
twitter_df['tweet_text'] = twitter_df[text_columns].fillna('').apply(lambda x: ' '.join(x), axis=1)

# Rename important columns with comments
twitter_df = twitter_df.rename(columns={
    'css-146c3p1': 'date',                # Posting date of the tweet
    'css-1jxf684 8': 'likes',             # Number of likes
    'css-1jxf684 11': 'retweets',         # Number of retweets
    'css-1jxf684 5': 'keyword_from_tweets'  # Extracted keyword/topic
})


In [47]:
# Step 8.1: Clean tweet_text (remove usernames, first word, bullet symbols, URLs, extra spaces)
import re

def clean_tweet(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r'@\w+', '', text)        # Remove @usernames
    text = re.sub(r'·', '', text)           # Remove bullet symbol
    text = re.sub(r'http\S+', '', text)     # Remove URLs
    words = text.split()
    text = ' '.join(words[1:]) if len(words) > 1 else text  # Remove first word (username-like)
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

twitter_df['tweet_text'] = twitter_df['tweet_text'].apply(clean_tweet)


In [48]:
# Step 9: Keep only the required columns from the DataFrame
#  want to keep: tweet text, date, likes, retweets, and keywords
twitter_cleaned = twitter_df[['tweet_text', 'date', 'likes', 'retweets', 'keyword_from_tweets']]

# Save the cleaned DataFrame into a new CSV file
twitter_cleaned.to_csv("twitter_cleaned.csv", index=False)  


In [49]:
# Step 10: Show final shape and preview
print("\nFinal Shape:", twitter_cleaned.shape)
print("\nCleaned & Filtered Sample:")
print(twitter_cleaned.head(3))


Final Shape: (1550, 5)

Cleaned & Filtered Sample:
                                          tweet_text    date likes retweets  \
0  Canada faces an economic crossroads: - Unemplo...  May 14    59      39K   
1  Donovan Canada's unemployment rate has now hit...   Jun 8   388     117K   
2  BREAKING CANADIAN YOUTH 139 696 68K UNEMPLOYME...  Jun 26   139      68K   

  keyword_from_tweets  
0        Unemployment  
1        unemployment  
2            CANADIAN  


In [37]:
# Step 10: Save the cleaned DataFrame to CSV
twitter_cleaned.to_csv("twitter_cleaned.csv", index=False)


## Preprocessing Labour stats data 

In [5]:
# Load dataset
employment_df = pd.read_csv('Employment by class of worker and industry, seasonally adjusted.csv', skiprows=1)
labour_df = pd.read_csv('Labour force characteristics by province, seasonally adjusted.csv', skiprows=1)


In [6]:
# -------------------- Employment by class of worker and industry DATA CLEANING --------------------
print("=== Employment Data ===")
print("Initial Shape:", employment_df.shape)
print("\nNull values:\n", employment_df.isnull().sum())


=== Employment Data ===
Initial Shape: (33, 8)

Null values:
                            3
 April 2025                9
 May 2025                  9
 Standard error{1}         9
 April to May 2025         9
 May 2024 to May 2025      9
 April to May 2025.1       9
 May 2024 to May 2025.1    9
dtype: int64


In [7]:
#Step 2: Check null & missing values
print("\nNull values per column:")
print(employment_df.isnull().sum())


Null values per column:
                           3
 April 2025                9
 May 2025                  9
 Standard error{1}         9
 April to May 2025         9
 May 2024 to May 2025      9
 April to May 2025.1       9
 May 2024 to May 2025.1    9
dtype: int64


In [8]:
print(employment_df.isnull().mean()*100)

                            9.090909
 April 2025                27.272727
 May 2025                  27.272727
 Standard error{1}         27.272727
 April to May 2025         27.272727
 May 2024 to May 2025      27.272727
 April to May 2025.1       27.272727
 May 2024 to May 2025.1    27.272727
dtype: float64


In [9]:
print([col for col in employment_df.columns])


['\xa0', '\xa0April 2025', '\xa0May 2025', '\xa0Standard error{1}', '\xa0April to May 2025', '\xa0May 2024 to May 2025', '\xa0April to May 2025.1', '\xa0May 2024 to May 2025.1']


In [10]:
employment_df.columns = employment_df.columns.str.strip()
print(employment_df.columns)


Index(['', 'April 2025', 'May 2025', 'Standard error{1}', 'April to May 2025',
       'May 2024 to May 2025', 'April to May 2025.1',
       'May 2024 to May 2025.1'],
      dtype='object')


In [11]:
employment_df.rename(columns={
    '': 'class_of_worker',
    'April 2025': 'april_2025',
    'May 2025': 'may_2025',
    'Standard error{1}': 'standard_error',
    'April to May 2025': 'change_apr_may_2025',
    'May 2024 to May 2025': 'change_may2024_may2025',
    'April to May 2025.1': 'pct_change_apr_may_2025',
    'May 2024 to May 2025.1': 'pct_change_may2024_may2025'
}, inplace=True)
print(employment_df.columns)


Index(['class_of_worker', 'april_2025', 'may_2025', 'standard_error',
       'change_apr_may_2025', 'change_may2024_may2025',
       'pct_change_apr_may_2025', 'pct_change_may2024_may2025'],
      dtype='object')


In [12]:
# Drop rows where all values are NaN
employment_df.dropna(how='all', inplace=True)


In [13]:
print(employment_df.columns)


Index(['class_of_worker', 'april_2025', 'may_2025', 'standard_error',
       'change_apr_may_2025', 'change_may2024_may2025',
       'pct_change_apr_may_2025', 'pct_change_may2024_may2025'],
      dtype='object')


In [14]:
# Convert columns to numeric:
for col in employment_df.columns[1:]:
    employment_df[col] = pd.to_numeric(employment_df[col], errors='coerce')


In [15]:
# Fill NaN with median:
numeric_cols = employment_df.select_dtypes(include=['float64', 'int64']).columns
employment_df[numeric_cols] = employment_df[numeric_cols].fillna(employment_df[numeric_cols].median())


In [16]:
# Fill the first column (class_of_worker) with forward fill:
employment_df['class_of_worker'] = employment_df['class_of_worker'].fillna(method='ffill')


C:\Users\birva\AppData\Local\Temp\ipykernel_14256\3484244376.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  employment_df['class_of_worker'] = employment_df['class_of_worker'].fillna(method='ffill')


In [17]:
employment_df.isnull().sum().sum()


1

In [32]:

# Remove rows that contain keywords like "Note", "Source", "Table"
employment_df = employment_df[~employment_df['class_of_worker'].str.contains("Note|Source|Table|rounding", case=False, na=False)]

In [33]:
employment_df['class_of_worker'].fillna('Unknown', inplace=True)


C:\Users\birva\AppData\Local\Temp\ipykernel_14256\1135438297.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  employment_df['class_of_worker'].fillna('Unknown', inplace=True)


In [34]:
print(employment_df.isnull().mean()*100)

class_of_worker               0.0
april_2025                    0.0
may_2025                      0.0
standard_error                0.0
change_apr_may_2025           0.0
change_may2024_may2025        0.0
pct_change_apr_may_2025       0.0
pct_change_may2024_may2025    0.0
dtype: float64


In [35]:
employment_df.to_csv("employment_cleaned.csv", index=False)


In [110]:
## -------------------- Labour force characteristics by province DATA CLEANING --------------------
print("=== Labour force characteristics by province, seasonally adjusted ===")
print("Initial Shape:", labour_df.shape)
print("\nNull values:\n", labour_df.isnull().sum())


=== Labour force characteristics by province, seasonally adjusted ===
Initial Shape: (117, 8)

Null values:
                            11
 April 2025                26
 May 2025                  26
 Standard error{1}         26
 April to May 2025         26
 May 2024 to May 2025      26
 April to May 2025.1       26
 May 2024 to May 2025.1    26
dtype: int64


In [112]:
print(labour_df.isnull().mean()*100)

                            9.401709
 April 2025                22.222222
 May 2025                  22.222222
 Standard error{1}         22.222222
 April to May 2025         22.222222
 May 2024 to May 2025      22.222222
 April to May 2025.1       22.222222
 May 2024 to May 2025.1    22.222222
dtype: float64


In [113]:
print([col for col in labour_df.columns])

['\xa0', '\xa0April 2025', '\xa0May 2025', '\xa0Standard error{1}', '\xa0April to May 2025', '\xa0May 2024 to May 2025', '\xa0April to May 2025.1', '\xa0May 2024 to May 2025.1']


In [116]:
labour_df.columns = labour_df.columns.str.strip()
print(labour_df.columns)


Index(['', 'April 2025', 'May 2025', 'Standard error{1}', 'April to May 2025',
       'May 2024 to May 2025', 'April to May 2025.1',
       'May 2024 to May 2025.1'],
      dtype='object')


In [117]:
labour_df.rename(columns={
    ' ': 'province',
    'April 2025': 'april_2025',
    'May 2025': 'may_2025',
    'Standard error{1}': 'standard_error',
    'April to May 2025': 'change_apr_may_2025',
    'May 2024 to May 2025': 'change_may2024_may2025',
    'April to May 2025.1': 'pct_change_apr_may_2025',
    'May 2024 to May 2025.1': 'pct_change_may2024_may2025'
}, inplace=True)


In [119]:
 ## Drop rows where all values are NaN
labour_df.dropna(how='all', inplace=True)


In [121]:
 ## Fill text column (province) using forward fill
if 'province' in labour_df.columns:
    labour_df['province'] = labour_df['province'].fillna(method='ffill')
    labour_df['province'] = labour_df['province'].fillna('Unknown')


In [122]:
#  Convert numeric columns to float and fill with median
for col in labour_df.columns[1:]:
    labour_df[col] = pd.to_numeric(labour_df[col], errors='coerce')

numeric_cols = labour_df.select_dtypes(include=['float64', 'int64']).columns
labour_df[numeric_cols] = labour_df[numeric_cols].fillna(labour_df[numeric_cols].median())


In [123]:
print(labour_df.isnull().mean()*100)

                              0.934579
april_2025                    0.000000
may_2025                      0.000000
standard_error                0.000000
change_apr_may_2025           0.000000
change_may2024_may2025        0.000000
pct_change_apr_may_2025       0.000000
pct_change_may2024_may2025    0.000000
dtype: float64


In [124]:
 ## Save the cleaned CSV
labour_df.to_csv('labour_force_cleaned.csv', index=False)

print("Labour force data cleaned and saved to labour_force_cleaned.csv")

Labour force data cleaned and saved to labour_force_cleaned.csv
